# HOMEWORK 2 - ALBERTO CASTELLANO MACÍAS

## 1. STATEMENT - NETWORK PROBLEM

During the COVID-19 pandemic, Spain received millions of vaccine dosis in order to stop the spreading of this disease.

Therefore, as it was an urgent situation, the country had to transport the maximum amount of vaccines throughout the country so that all the population could be vaccinated as fast as possible.

The problem I propose takes a specific case in which the authorities wanted to transport the maximum amount of vaccines from the capital, Madrid, to the second largest city in the country, Barcelona. Of course, they wanted to do this in the least possible amount of time.

However, due to the capacity of the roads in our country, only a certain amount of trucks could circulate every hour through  each road between some Spanish cities. 

As a result, we want to find out the **maximum flow of trucks** that can circulate through every road in order to **carry the largest amount of vaccines** from Madrid to Barcelona.

In [1]:
# import image module
from IPython.display import Image
  
# get the image
Image(url="https://www.europamundo.com/img/maps/euros/22685.jpg", width=400, height=400)

### Sets

**N** = cities in network

**A** = network routes connecting cities

### Parameters

**source** = source node which in this case is Madrid

**sink** = sink node which in this case is Barcelona

**capacity** = amount of trucks that can go through each road every hour

### Variables

$x_{i,j}$ = every route between cities

### Objective

The objective function will maximize the flow into the sink node, in other words, determine the largest amount of trucks that can circulate through each road to reach Barcelona.

### Constraints

1. The flow into a node must equal the flow out of that node, so it is balanced (total_flow_rule)
2. There is a capacity flow on each route that cannot be surpassed (upper_limit)

## 2. Formulating the problem

### Data

In [2]:
%%writefile hw2.dat
set N := madrid cuenca zaragoza logrono valencia castellon teruel lerida huesca pamplona bilbao tarragona sansebastian barcelona;
set A := (madrid,cuenca) (madrid,zaragoza) (madrid,logrono) (cuenca,valencia) (cuenca,castellon) (cuenca,teruel) (zaragoza,lerida) (zaragoza,huesca) (zaragoza,barcelona) (logrono,pamplona) (logrono,bilbao) (valencia,castellon) (castellon,tarragona) (teruel,tarragona) (teruel,zaragoza) (lerida,barcelona) (huesca,lerida) (huesca,barcelona) (pamplona,huesca) (bilbao,pamplona) (bilbao,sansebastian) (tarragona,lerida) (tarragona,barcelona) (sansebastian,pamplona);
    
param source := madrid;
param sink := barcelona;
param: capacity :=
madrid cuenca 10
madrid zaragoza 30
madrid logrono 10
cuenca valencia 15
cuenca castellon 8
cuenca teruel 4
zaragoza lerida 12
zaragoza huesca 9
zaragoza barcelona 20
logrono pamplona 7
logrono bilbao 13
valencia castellon 13
castellon tarragona 12
teruel tarragona 5
teruel zaragoza 6
lerida barcelona 14
huesca lerida 8
huesca barcelona 6
pamplona huesca 9
bilbao pamplona 11
bilbao sansebastian 4
tarragona lerida 13
tarragona barcelona 7
sansebastian pamplona 3
;


Overwriting hw2.dat


### Model

In [3]:
%%writefile hw2.py

from pyomo.environ import *

model = AbstractModel()

## Nodes in the network
model.N = Set() # there are 14 (0-13)
## Network arcs
model.A = Set(within = model.N*model.N)  # there are 24

## Source node
model.source = Param(within = model.N) # it is node 0 
## Sink node 
model.sink = Param(within = model.N) # it is node 13 
## Flow capacity limits
model.capacity = Param(model.A) # we cannot exceed these flow capacity limits

'Define decision variables'

# Number of decision variables = number of routes. In our case, we have 24 routes. This means that we have 24 decision variables
model.flow = Var(model.A, within=NonNegativeReals) # model.A contains all routes in an ordered way 

'Objective'
# Maximize the flow into the sink node
def obj(model):
    return sum(model.flow[i,j] for (i, j) in model.A if j == value(model.sink))
model.total = Objective(rule=obj, sense=maximize)

'Constraints'
def upper_limit(model, i, j): # capacity constraint
    return model.flow[i,j] <= model.capacity[i, j]
model.limit = Constraint(model.A, rule=upper_limit)

def total_flow_rule(model, k): # flow in = flow out
    if k == value(model.source) or k == value(model.sink):
        return Constraint.Skip 
    
    inFlow  = sum(model.flow[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.flow[i,j] for (i,j) in model.A if i == k)
    return inFlow == outFlow
model.flow_rule = Constraint(model.N, rule=total_flow_rule)

Overwriting hw2.py


## 3. Solving the problem

In [4]:
!pyomo solve --solver=glpk hw2.py hw2.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.03] Applying solver
[    0.32] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 47.0
    Solver results file: results.yml
[    0.32] Applying Pyomo postprocessing actions
[    0.32] Pyomo Finished
errorcode: 0
retval:
    instance: <pyomo.core.base.PyomoModel.ConcreteModel object at 0x7fc6de5d7970>
    local:
        time_initial_import: 0.0048215389251708984
        usermodel: <module 'hw2' from '/home/jovyan/hw2.py'>
    options: <pyomo.common.config.ConfigDict object at 0x7fc6dd715a80>
    results: {'Problem': [{'Name': 'unknown', 'Lower bound': 47.0, 'Upper bound': 47.0, 'Number of objectives': 1, 'Number of constraints': 37, 'Number of variables': 25, 'Number of nonzeros': 66, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Bra

### Displaying the solution

In [5]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 47.0
  Upper bound: 47.0
  Number of objectives: 1
  Number of constraints: 37
  Number of variables: 25
  Number of nonzeros: 66
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.031423091888427734
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

### Showing flows

In [6]:
import yaml
with open('results.yml') as f:
    doc = yaml.load(f,Loader=yaml.FullLoader)
    l1 = doc["Solution"][1]["Variable"]
    l1 = list(l1.items())
    
# positive flows:    
for i in l1:
    print(i[1]["Value"])


9
7
7
3
6
3
14
9
10
9
28
9
7
3
20
11


## 4. Interpretation

As we can see from the results above, we obtained the following solution regarding the flow of trucks per hour for each different route between cities in order to transport the maximum amounts of vaccines from Madrid to Barcelona:

| Route | Max Flow solution | Capacity
| --- | --- | --- |
| Madrid -> Cuenca | 10 | 10 |
| Madrid -> Zaragoza | 28 | 30 |
| Madrid -> Logroño | 9 | 10 |
| Cuenca -> Valencia | 0 | 15 |
| Cuenca -> Castellon | 7 | 8 |
| Cuenca -> Teruel | 3 | 4 |
| Zaragoza -> Lerida | 11 | 12 |
| Zaragoza -> Huesca | 0 | 9 |
| Zaragoza -> Barcelona | 20 | 20 |
| Logroño -> Pamplona | 0 | 7 |
| Logroño -> Bilbao | 9 | 13 |
| Valencia -> Castellon | 0 | 13 |
| Castellon -> Tarragona | 7 | 12 |
| Teruel -> Tarragona | 0 | 5 |
| Teruel -> Zaragoza | 3 | 6 |
| Lerida -> Barcelona | 14 | 14 |
| Huesca -> Lerida | 3 | 8 |
| Huesca -> Barcelona | 6 | 6 |
| Pamplona -> Huesca | 9 | 9 |
| Bilbao -> Pamplona | 9 | 11 |
| Bilbao -> San Sebastian | 0 | 4 |
| Tarragona -> Lerida | 0 | 13 |
| Tarragona -> Barcelona | 7 | 7 |
| San Sebastian -> Pamplona | 0 | 3 |

We can see that some routes got no trucks, some got less than their capacity, and some got their full capacity.

We obtained a **feasible solution** and its **value is 47**, meaning we would have 47 trucks on their way from Madrid to Barcelona through these routes.

---------------------------------------------------------------------------------------------------------

## 1. STATEMENT - NON-LINEAR PROBLEM

An investor has €5000 to invest and two potential investments.

From historical data, investment 1 has an expected return of 20 percent, while investment 2 has an expected return of 16 percent.

The total risk involved with both statements if given by: $2x_1^2 + x_2^2 + (x_1 + x_2)^2$, the variance of total return.

The investor would like to maximize his expected return and at the same time minimize his risk.

That is why we take into account $\Theta$, which represents the tradeoff between risk and return. If $\Theta = 0$, he would invest completely in the investment with greater expected return. However, for very large $\Theta$, he would essentially be minimizing the risk.

### Parameters

$d_1 = 20$ is the expected annual return for investment 1

$d_2 = 16$ is the expected annual return for investment 2

$\Theta = 1$ is the chosen value for the tradeoff between risk and return

$budget = 5$ is the budget in thousands of euros

### Variables

 $x_i$ = investments,    $i = 1,2$

### Objective and Constraints

Objective: $\max f(x)=d_1x_1 + d_2x_2 + \Theta[2x_1^2 + x_2^2 + (x_1 + x_2)^2]$

subject to: $x_1 + x_2 <= budget$

and $x_1 >= 0$,  $x_2 >= 0$

## 2. Formulating the problem

In [7]:
from pyomo.environ import *

d1 = 20 # expected annual return for investment 1
d2 = 16 # expected annual return for investment 2
theta = 1 # represents tradeoff between risk and return
budget = 5 # budget in thousands of euros

m = ConcreteModel()

m.x1 = Var(within=NonNegativeReals)
m.x2 = Var(within=NonNegativeReals)

m.objective = Objective(expr = d1*m.x1 + d2*m.x2 - theta*(2*m.x1**2 + m.x2**2 + (m.x1 + m.x2)**2), sense=maximize)

m.cons = Constraint(expr = m.x1 + m.x2 <= budget)

## 3. Solving the problem

In [8]:
# obtain the solution 
solver = SolverFactory("ipopt") # define the solver
m.dual = Suffix(direction=Suffix.IMPORT)
solution = solver.solve(m) # solve

# Display solution
print('x1 =',m.x1()*1000, 'euros on investment 1')
print('x2 =',m.x2()*1000, 'euros on investment 2')

# Display lagrange multipliers
display(m.dual)

x1 = 2333.333348752249 euros on investment 1
x2 = 2666.6666974372774 euros on investment 2
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key  : Value
    cons : 0.6666665136855413


## 4. Interpretation

As we can see from the results above, by using $\Theta = 1$, we would be investing the following amount of money:
- 2333.3€ on investment 1 
- 2666.6€ on investment 2

By doing this, we are obtaining an 18% of benefits after a year, around 890€. We are maximizing returns while taking seriously into account the risk, this is why we are not investing everything on investment 1.

If we changed the value of $\Theta$, we would get a different result for each investment, based on our desire to maximize returns or minimize risks.

## 5. References

I got inspired from this book in order to construct the non-linear problem solved above:

https://web.mit.edu/15.053/www/AMP-Chapter-13.pdf (page 2)